In [ ]:
import re
from datasets import load_dataset

In [ ]:
# brazilian alphabet
lower_case = r'abcdefghijklmnopqrstuvwxyzáàâãéêíóôõúç'
upper_case = r'ABCDEFGHIJKLMNOPQRSTUVWXYZÁÀÂÃÉÊÍÓÔÕÚÇ'

In [ ]:
dt = load_dataset('carolmou/random-sentences')

In [ ]:
freq = {}
test_sentences = []

In [ ]:
# matches all lower case words or word with the first upper character and hiphenized words
reg = rf'\b(?:[{upper_case}][{lower_case}]*|[{lower_case}]+(?:-[{lower_case}]+)*|[{lower_case}]*[{upper_case}](?=[{lower_case}]))\b'

In [ ]:
for type in ["train", "test"]:
    # 'type' dictionary of frequencies
    dic = {}

    wrong = dt[type]["wrong_text"]
    correct = dt[type]["correct_text"]

    for w_sentence, c_sentence in zip(wrong, correct):
        # find all words of each sentence
        w_words = list(re.findall(reg, w_sentence))
        c_words = list(re.findall(reg, c_sentence))

        tuples = []

        # iterate through them pairwise
        for w1, w2 in zip(w_words, c_words):
            if w1 == w2:
                continue

            # they're different; this is where
            # the model has to act
            dic[(w1, w2)] = dic.get((w1, w2), 0) + 1

            # for each test sentence, make a list
            # of the different pairs
            if type == "test":
                tuples.append((w1, w2))

        if type == "test":
            test_sentences.append(tuples)

    # retrieve alias
    freq[type] = dic

In [ ]:
# preview for sanity
freq["test"]

In [ ]:
# amount of sentences where every mistake
# has been seen in the train before
qtd = 0

for sentence in test_sentences:
    for tup in sentence:
        # the model has never seen this mistake
        # before; it has to extrapolate
        if freq['train'].get(tup,0) == 0:
            break
    else:
        qtd += 1

In [ ]:
total_sentences = len(dt['test']['wrong_text'])

# percentage of bad sentences in the test set
print((qtd/total_sentences)*100)